In [180]:
from pathlib import Path
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import lit,to_timestamp, coalesce, col, date_format, when
from functools import reduce
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import warnings

from sqlalchemy import create_engine, text
import pymysql


spark = SparkSession.builder.getOrCreate()

In [181]:
# Define la ruta base
base_path = Path("/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS")

# Lista 1: Archivos .csv que contienen "AQ_SEP" en su nombre
lista_aq_sep = [archivo for archivo in base_path.rglob("*.csv") if "AQ_SEP" in archivo.name]

# Lista 2: Archivos .csv que contienen "AirQualityUnit01" en su nombre
lista_airqualityunit01 = [archivo for archivo in base_path.rglob("*.csv") if "AirQualityUnit01" in archivo.name]

# Lista 3: Archivos .csv que contienen "AirQualityUnit02" en su nombre
lista_airqualityunit02 = [archivo for archivo in base_path.rglob("*.csv") if "AirQualityUnit02" in archivo.name]


In [182]:
# Imprimir las listas archivos AQ_SEP
print("Archivos con 'AQ_SEP':")
for archivo in lista_aq_sep:
    print(archivo)

Archivos con 'AQ_SEP':
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Abril 2024/6etmrenvironmentobserved - AQ_SEP-data-2024-04-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Junio 2024/6etmrenvironmentobserved - AQ_SEP, activa desde el 1ero de mayo-data-2024-06-14.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Agosto 2024/6etmrenvironmentobserved - AQ_SEP, activa desde el 1ero de mayo-data-2024-08-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Febrero 2024/6etmrenvironmentobserved - AQ_SEP-data-2024-02-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Diciembre 2023/6etmrenvironmentobserved - AQ_SEP-data-2023-12-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Marzo 2024/6etmrenvironmentobserved - AQ_SEP-data-2024-03-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Noviembre 2023/6etmrenvironmentobserved - AQ_SEP-data-2023-11-15.

In [183]:
# Imprimir las listas archivos AirQualityUnit01
print("\nArchivos con 'AirQualityUnit01':")
for archivo in lista_airqualityunit01:
    print(archivo)


Archivos con 'AirQualityUnit01':
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Abril 2024/7etmrairqualityobserved - AirQualityUnit01-data-2024-04-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Junio 2024/7etmrairqualityobserved - AirQualityUnit01-data-2024-06-14.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Julio 2024/7etmrairqualityobserved - AirQualityUnit01-data-2024-07-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Agosto 2024/7etmrairqualityobserved - AirQualityUnit01-data-2024-08-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Febrero 2024/7etmrairqualityobserved - AirQualityUnit01-data-2024-02-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Diciembre 2023/7etmrairqualityobserved - AirQualityUnit01-data-2023-12-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Marzo 2024/7etmrairqualityobserved - AirQualityUnit01-data-2

In [184]:
# Imprimir las listas archivos AirQualityUnit02
print("\nArchivos con 'AirQualityUnit02':")
for archivo in lista_airqualityunit02:
    print(archivo)


Archivos con 'AirQualityUnit02':
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Abril 2024/8etmrairqualityobserved - AirQualityUnit02-data-2024-04-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Junio 2024/8etmrairqualityobserved - AirQualityUnit02-data-2024-06-14.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Agosto 2024/8etmrairqualityobserved - AirQualityUnit02-data-2024-08-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Febrero 2024/8etmrairqualityobserved - AirQualityUnit02-data-2024-02-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Diciembre 2023/8etmrairqualityobserved - AirQualityUnit02-data-2023-12-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Marzo 2024/8etmrairqualityobserved - AirQualityUnit02-data-2024-03-15.csv
/home/steven-laptop/Documentos/Estudio/Proyecto QAIR/datos/DATOS/Noviembre 2023/8etmrairqualityobserved - AirQualityUnit02-da

In [185]:
def detect_separator_simple(file_path, sample_size=1024):
    """
    Detecta el separador de un archivo CSV leyendo una muestra del archivo.

    Args:
        file_path (str): Ruta del archivo CSV.
        sample_size (int, optional): Número de bytes a leer para el análisis. Por defecto es 1024.

    Returns:
        str: El caracter delimitador detectado o None si no se pudo detectar.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            sample = f.read(sample_size)
        dialect = csv.Sniffer().sniff(sample)
        return dialect.delimiter
    except Exception as e:
        print(f"Error al detectar el separador: {e}")
        return None


In [186]:
def load_csv_correctly(path):
    # Leer el archivo como texto
    raw_df = spark.read.text(path)
    # Obtener la primera línea
    first_line = raw_df.first()[0]
    separator = detect_separator_simple(path)
    
    if first_line and isinstance(first_line, str) and first_line.startswith("sep="):
        # Omitir la primera línea usando zipWithIndex para mantener el orden
        lines_rdd = raw_df.rdd.zipWithIndex() \
                        .filter(lambda row_index: row_index[1] > 0) \
                        .map(lambda row_index: row_index[0][0])
        
        # Leer el CSV a partir del RDD de líneas
        df = spark.read.option("header", "true") \
                       .option("inferSchema", "true") \
                       .option("escape", '"') \
                       .option("quote", '"') \
                       .option("sep", separator) \
                       .csv(lines_rdd)
    else:
        # Leer el CSV normalmente desde la ruta
        df = spark.read.option("header", "true") \
                       .option("inferSchema", "true") \
                       .option("escape", '"') \
                       .option("quote", '"') \
                       .option("sep", separator) \
                       .csv(path)
    
    return df


In [187]:
DF_LIST_AQ_SEP = [load_csv_correctly(str(path)) for path in lista_aq_sep]
DF_LIST_AQ_01 = [load_csv_correctly(str(path)) for path in lista_airqualityunit01]
DF_LIST_AQ_02 = [load_csv_correctly(str(path)) for path in lista_airqualityunit02]

In [188]:
# Función para limpiar los nombres de columnas
def clean_column_names(df):
    new_columns = [col.strip().replace(" ", "_").lower() for col in df.columns]
    return df.toDF(*new_columns) 

In [189]:
DF_LIST_AQ_SEP = [clean_column_names(df) for df in DF_LIST_AQ_SEP]
DF_LIST_AQ_01 = [clean_column_names(df) for df in DF_LIST_AQ_01]
DF_LIST_AQ_02 = [clean_column_names(df) for df in DF_LIST_AQ_02]

In [190]:
for i, df in enumerate(DF_LIST_AQ_SEP):
    print(f"DataFrame DF_LIST_AQ_SEP {i} columns: {df.columns}")

DataFrame DF_LIST_AQ_SEP 0 columns: ['time', 'co_aq_sep', 'no2_aq_sep', 'o3_aq_sep', 'pm1_aq_sep', 'pm10_aq_sep', 'pm2_5_aq_sep', 'pressure_aq_sep', 'relativehumidity_aq_sep', 'temperature_aq_sep']
DataFrame DF_LIST_AQ_SEP 1 columns: ['time', 'co_aq_sep', 'humidity_aq_sep', 'o3_aq_sep', 'pm1_aq_sep', 'pm10_aq_sep', 'pm2_5_aq_sep', 'pressure_aq_sep', 'temperature_aq_sep']
DataFrame DF_LIST_AQ_SEP 2 columns: ['time', 'co_aq_sep', 'humidity_aq_sep', 'o3_aq_sep', 'pm1_aq_sep', 'pm10_aq_sep', 'pm2_5_aq_sep', 'pressure_aq_sep', 'temperature_aq_sep']
DataFrame DF_LIST_AQ_SEP 3 columns: ['time', 'co_aq_sep', 'no2_aq_sep', 'o3_aq_sep', 'pm1_aq_sep', 'pm10_aq_sep', 'pm2_5_aq_sep', 'pressure_aq_sep', 'relativehumidity_aq_sep', 'temperature_aq_sep']
DataFrame DF_LIST_AQ_SEP 4 columns: ['time', 'co_aq_sep', 'no2_aq_sep', 'o3_aq_sep', 'pm1_aq_sep', 'pm10_aq_sep', 'pm2_5_aq_sep', 'pressure_aq_sep', 'relativehumidity_aq_sep', 'temperature_aq_sep']
DataFrame DF_LIST_AQ_SEP 5 columns: ['time', 'co_aq_se

In [191]:
for i, df in enumerate(DF_LIST_AQ_01):
    print(f"DataFrame DF_LIST_AQ_01 {i} columns: {df.columns}")

DataFrame DF_LIST_AQ_01 0 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_01 1 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_01 2 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_01 3 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_01 4 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_01 5 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_01 6 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFr

In [192]:
for i, df in enumerate(DF_LIST_AQ_02):
    print(f"DataFrame DF_LIST_AQ_02 {i} columns: {df.columns}")

DataFrame DF_LIST_AQ_02 0 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_02 1 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_02 2 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_02 3 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_02 4 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_02 5 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFrame DF_LIST_AQ_02 6 columns: ['time', 'entity_id', 'co', 'co2', 'o3', 'pm10', 'pm2_5', 'pm5', 'relativehumidity', 'so2', 'temperature']
DataFr

In [193]:
def standardize_df(df):
    # Lista de columnas deseadas en el orden y con los nombres estándar
    desired_order = ["time", "pm2_5", "pm10", "co", "o3", "humidity", "temperature","pm5"]
    new_columns = []
    
    # Para cada columna deseada, buscar en el DataFrame original la primera columna que contenga la palabra clave
    for keyword in desired_order:
        # Buscar candidates: columnas cuyo nombre (en minúscula) contenga la palabra clave
        candidates = [col for col in df.columns if keyword in col.lower()]
        if candidates:
            # Si existen varias, se toma la primera (podrías ajustar esto si necesitas otro criterio)
            new_columns.append(F.col(candidates[0]).alias(keyword))
        else:
            # Si no existe la columna, se agrega una columna con valores nulos
            new_columns.append(F.lit(None).alias(keyword))
    
    # Seleccionar y devolver el nuevo DataFrame con las columnas en el orden deseado
    return df.select(*new_columns)

In [194]:
DF_LIST_AQ_SEP_S = [standardize_df(df) for df in DF_LIST_AQ_SEP]
TOTAL_LIST_AQ_SEP_S = 0
for i, df in enumerate(DF_LIST_AQ_SEP_S):
    print(f"DataFrame DF_LIST_AQ_SEP_S {i} columns: {df.columns} count: {df.count()}")
    TOTAL_LIST_AQ_SEP_S += df.count()
print(f"Total de filas en todos los DataFrames: {TOTAL_LIST_AQ_SEP_S}")
DF_LIST_AQ_SEP_S[0].show()

DataFrame DF_LIST_AQ_SEP_S 0 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 2927
DataFrame DF_LIST_AQ_SEP_S 1 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 1525
DataFrame DF_LIST_AQ_SEP_S 2 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 1591
DataFrame DF_LIST_AQ_SEP_S 3 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 2866
DataFrame DF_LIST_AQ_SEP_S 4 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 2765
DataFrame DF_LIST_AQ_SEP_S 5 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 2657
DataFrame DF_LIST_AQ_SEP_S 6 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 2959
DataFrame DF_LIST_AQ_SEP_S 7 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 2859
DataFrame DF_LIST_AQ_SEP

In [195]:
DF_LIST_AQ_01_S = [standardize_df(df) for df in DF_LIST_AQ_01]
TOTAL_LIST_AQ_01_S = 0
for i, df in enumerate(DF_LIST_AQ_01_S):
    print(f"DataFrame DF_LIST_AQ_01_S {i} columns: {df.columns} count: {df.count()}")
    TOTAL_LIST_AQ_01_S += df.count()
print(f"Total de filas en todos los DataFrames: {TOTAL_LIST_AQ_01_S}")
DF_LIST_AQ_01_S[0].show()

DataFrame DF_LIST_AQ_01_S 0 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 2841
DataFrame DF_LIST_AQ_01_S 1 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 12962
DataFrame DF_LIST_AQ_01_S 2 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 14216
DataFrame DF_LIST_AQ_01_S 3 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 14879
DataFrame DF_LIST_AQ_01_S 4 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 14376
DataFrame DF_LIST_AQ_01_S 5 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 14168
DataFrame DF_LIST_AQ_01_S 6 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 13346
DataFrame DF_LIST_AQ_01_S 7 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 15215
DataFrame DF_LIST_AQ_01_S

In [196]:
DF_LIST_AQ_02_S = [standardize_df(df) for df in DF_LIST_AQ_02]
TOTAL_LIST_AQ_02_S = 0
for i, df in enumerate(DF_LIST_AQ_02_S):
    print(f"DataFrame DF_LIST_AQ_02_S {i} columns: {df.columns} count: {df.count()}")
    TOTAL_LIST_AQ_02_S += df.count()
print(f"Total de filas en todos los DataFrames: {TOTAL_LIST_AQ_02_S}")
DF_LIST_AQ_02_S[0].show()

DataFrame DF_LIST_AQ_02_S 0 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 2803
DataFrame DF_LIST_AQ_02_S 1 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 6471
DataFrame DF_LIST_AQ_02_S 2 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 3564
DataFrame DF_LIST_AQ_02_S 3 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 14385
DataFrame DF_LIST_AQ_02_S 4 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 13991
DataFrame DF_LIST_AQ_02_S 5 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 13471
DataFrame DF_LIST_AQ_02_S 6 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 14786
DataFrame DF_LIST_AQ_02_S 7 columns: ['time', 'pm2_5', 'pm10', 'co', 'o3', 'humidity', 'temperature', 'pm5'] count: 14245
DataFrame DF_LIST_AQ_02_S 8

In [197]:
# Combinar todos los DataFrames en uno solo - DF_LIST_AQ_SEP_S
DF_AQ_SEP = reduce(lambda df1, df2: df1.unionByName(df2), DF_LIST_AQ_SEP_S)
print(f"Total de filas en todos los DF_AQ_SEP: {DF_AQ_SEP.count()}")
DF_AQ_SEP.show()

Total de filas en todos los DF_AQ_SEP: 32172
+-------------------+-----+----+----+---+--------+-----------+----+
|               time|pm2_5|pm10|  co| o3|humidity|temperature| pm5|
+-------------------+-----+----+----+---+--------+-----------+----+
|2024-03-15 00:11:34|  0.5| 0.5|25.3|0.1|  -999.0|       11.5|NULL|
|2024-03-15 00:26:34|  0.7| 0.7|23.0|0.1|  -999.0|       11.6|NULL|
|2024-03-15 00:41:34|  0.7| 0.7|22.2|0.1|  -999.0|       11.2|NULL|
|2024-03-15 00:56:34|  0.6| 0.6|20.4|0.1|  -999.0|       11.1|NULL|
|2024-03-15 01:11:34|  0.6| 0.6|19.6|0.1|  -999.0|       10.6|NULL|
|2024-03-15 01:26:34|  0.5| 0.5|19.2|0.1|  -999.0|       10.6|NULL|
|2024-03-15 01:41:34|  0.7| 0.7|19.5|0.1|  -999.0|       10.4|NULL|
|2024-03-15 01:56:34|  0.6| 0.6|21.2|0.1|  -999.0|       10.5|NULL|
|2024-03-15 02:11:34|  0.5| 0.5|20.3|0.1|  -999.0|       10.1|NULL|
|2024-03-15 02:26:34|  0.6| 0.6|19.2|0.1|  -999.0|        9.9|NULL|
|2024-03-15 02:41:34|  0.5| 0.5|19.7|0.1|  -999.0|        9.7|NULL|
|20

In [198]:
# Combinar todos los DataFrames en uno solo - DF_LIST_AQ_01_S
DF_AQ_1 = reduce(lambda df1, df2: df1.unionByName(df2), DF_LIST_AQ_01_S)
print(f"Total de filas en todos los DF_AQ_1: {DF_AQ_1.count()}")
DF_AQ_1.show()

Total de filas en todos los DF_AQ_1: 181913
+----------------+-----+----+----+----+--------+-----------+----+
|            time|pm2_5|pm10|  co|  o3|humidity|temperature| pm5|
+----------------+-----+----+----+----+--------+-----------+----+
|19/03/2024 22:08| -999|-999|-999|-999|  -999.0|     -998.0|-999|
| 01/04/2024 9:33| -999|-999|-999|-999|  -999.0|     -999.0|-999|
|08/04/2024 18:39|   44|  58|   2|-999|    60.3|       20.3|  29|
|08/04/2024 18:41|   46|  56|   3|-999|    60.3|       20.3|  30|
|08/04/2024 18:43|   46|  56|   4|-999|    60.3|       20.3|  30|
|08/04/2024 18:45|   46|  61|   5|-999|    60.4|       20.3|  28|
|08/04/2024 18:48|   45|  61|   6|-999|    60.5|       20.3|  27|
|08/04/2024 18:50|   45|  61|   7|-999|    60.6|       20.3|  27|
|08/04/2024 18:52|   46|  61|   8|-999|    60.7|       20.4|  28|
|08/04/2024 18:54|   44|  53|   9|-999|    60.7|       20.4|  28|
|08/04/2024 18:56|   44|  53|  10|-999|    60.7|       20.4|  28|
|08/04/2024 19:00|   43|  52|  1

In [199]:
# Combinar todos los DataFrames en uno solo - DF_LIST_AQ_02_S
DF_AQ_2 = reduce(lambda df1, df2: df1.unionByName(df2), DF_LIST_AQ_02_S)
print(f"Total de filas en todos los DF_AQ_2: {DF_AQ_2.count()}")
DF_AQ_2.show()

Total de filas en todos los DF_AQ_2: 148552
+----------------+-----+----+----+----+--------+-----------+----+
|            time|pm2_5|pm10|  co|  o3|humidity|temperature| pm5|
+----------------+-----+----+----+----+--------+-----------+----+
|19/03/2024 22:09| -999|-999|-999|-999|  -999.0|     -998.0|-999|
| 01/04/2024 9:33| -999|-999|-999|-999|  -999.0|     -999.0|-999|
|08/04/2024 20:18|   39|  45|   1|-999|    78.0|       19.0|  30|
|08/04/2024 20:20|   42|  54|   2|-999|    78.1|       19.3|  28|
|08/04/2024 20:46|   40|  47|   1|-999|    72.0|       20.3|  30|
|08/04/2024 20:48|   45|  59|   2|-999|    71.1|       20.4|  30|
|08/04/2024 20:53|   44|  56|   4|-999|    69.5|       20.5|  29|
|08/04/2024 20:55|   41|  51|   5|-999|    69.1|       20.5|  27|
|08/04/2024 20:57|   41|  52|   6|-999|    68.7|       20.6|  28|
|08/04/2024 20:59|   40|  52|   7|-999|    68.4|       20.6|  27|
|08/04/2024 21:05|   38|  48|  10|-999|    67.7|       20.8|  26|
|08/04/2024 21:07|   38|  51|  1

In [200]:
# Crear una condición que sea True cuando todas las columnas sean null

CONDITION = reduce(lambda acc, col: acc & F.col(col).isNull(), DF_AQ_2.columns, F.lit(True))

# Filtrar y contar las filas que cumplen la condición
DF_AQ_SEP_ALL_NULL = DF_AQ_SEP.filter(CONDITION).count()
DF_AQ_1_ALL_NULL = DF_AQ_1.filter(CONDITION).count()
DF_AQ_2_ALL_NULL = DF_AQ_2.filter(CONDITION).count()


print(" DF_AQ_SEP -> Cantidad de filas completamente null:", DF_AQ_SEP_ALL_NULL)
print(" DF_AQ_1 -> Cantidad de filas completamente null:", DF_AQ_1_ALL_NULL)
print(" DF_AQ_2 -> Cantidad de filas completamente null:", DF_AQ_2_ALL_NULL)

 DF_AQ_SEP -> Cantidad de filas completamente null: 0
 DF_AQ_1 -> Cantidad de filas completamente null: 0
 DF_AQ_2 -> Cantidad de filas completamente null: 0


In [201]:
def standardize_time_column(df):
    """
    Procesa la columna 'time' de un DataFrame y crea una nueva columna 'time_format'
    con el formato 'yyyy-MM-dd HH:mm:ss'.
    
    Considera varios formatos de fecha/hora:
      - dd/MM/yyyy H:mm       (Ej: 19/03/2024 22:09 o 01/04/2024 9:33)
      - yyyy-MM-dd H:mm:ss     (Ej: 2024-03-15 00:11:34)
      - yyyy-MM-dd H:mm        (Ej: 2024-03-15 11:34)
    """
    # Intenta convertir la columna "time" usando los diferentes formatos
    df = df.withColumn(
        "time_ts",
        coalesce(
            to_timestamp(col("time"), "dd/MM/yyyy H:mm"),
            to_timestamp(col("time"), "yyyy-MM-dd H:mm:ss"),
            to_timestamp(col("time"), "yyyy-MM-dd H:mm")
        )
    )
    
    # Formatea el timestamp al formato deseado y crea la nueva columna "time_format"
    df = df.withColumn("time_format", date_format(col("time_ts"), "yyyy-MM-dd HH:mm:ss"))
    
    # Elimina la columna intermedia si ya no es necesaria
    df = df.drop("time_ts")
    
    return df


In [202]:
#se crea una funcion que estandariza las fechas y agrega una nueva columna
DF_AQ_SEP = standardize_time_column(DF_AQ_SEP)
DF_AQ_1   = standardize_time_column(DF_AQ_1)
DF_AQ_2   = standardize_time_column(DF_AQ_2)


In [203]:
DF_AQ_SEP.show()
#DF_AQ_1.show()
#DF_AQ_2.show()

+-------------------+-----+----+----+---+--------+-----------+----+-------------------+
|               time|pm2_5|pm10|  co| o3|humidity|temperature| pm5|        time_format|
+-------------------+-----+----+----+---+--------+-----------+----+-------------------+
|2024-03-15 00:11:34|  0.5| 0.5|25.3|0.1|  -999.0|       11.5|NULL|2024-03-15 00:11:34|
|2024-03-15 00:26:34|  0.7| 0.7|23.0|0.1|  -999.0|       11.6|NULL|2024-03-15 00:26:34|
|2024-03-15 00:41:34|  0.7| 0.7|22.2|0.1|  -999.0|       11.2|NULL|2024-03-15 00:41:34|
|2024-03-15 00:56:34|  0.6| 0.6|20.4|0.1|  -999.0|       11.1|NULL|2024-03-15 00:56:34|
|2024-03-15 01:11:34|  0.6| 0.6|19.6|0.1|  -999.0|       10.6|NULL|2024-03-15 01:11:34|
|2024-03-15 01:26:34|  0.5| 0.5|19.2|0.1|  -999.0|       10.6|NULL|2024-03-15 01:26:34|
|2024-03-15 01:41:34|  0.7| 0.7|19.5|0.1|  -999.0|       10.4|NULL|2024-03-15 01:41:34|
|2024-03-15 01:56:34|  0.6| 0.6|21.2|0.1|  -999.0|       10.5|NULL|2024-03-15 01:56:34|
|2024-03-15 02:11:34|  0.5| 0.5|

In [145]:
# Filtrar las filas donde time_format es null
df_null_time = DF_AQ_SEP.filter(col("time_format").isNull())
# Mostrar las filas filtradas
df_null_time.show()
# Opcional: contar el número de filas con time_format null
null_count = DF_AQ_SEP.filter(col("time_format").isNull()).count()
print("DF_AQ_SEP - Número de filas con time_format null:", null_count)

+----+-----+----+---+---+--------+-----------+---+-----------+
|time|pm2_5|pm10| co| o3|humidity|temperature|pm5|time_format|
+----+-----+----+---+---+--------+-----------+---+-----------+
+----+-----+----+---+---+--------+-----------+---+-----------+

DF_AQ_SEP - Número de filas con time_format null: 0


In [146]:
# Filtrar las filas donde time_format es null
df_null_time = DF_AQ_1.filter(col("time_format").isNull())
# Mostrar las filas filtradas
df_null_time.show()
# Opcional: contar el número de filas con time_format null
null_count = DF_AQ_1.filter(col("time_format").isNull()).count()
print("DF_AQ_1 - Número de filas con time_format null:", null_count)

+----+-----+----+---+---+--------+-----------+---+-----------+
|time|pm2_5|pm10| co| o3|humidity|temperature|pm5|time_format|
+----+-----+----+---+---+--------+-----------+---+-----------+
+----+-----+----+---+---+--------+-----------+---+-----------+



DF_AQ_1 - Número de filas con time_format null: 0


In [147]:
# Filtrar las filas donde time_format es null
df_null_time = DF_AQ_2.filter(col("time_format").isNull())
# Mostrar las filas filtradas
df_null_time.show()
# Opcional: contar el número de filas con time_format null
null_count = DF_AQ_2.filter(col("time_format").isNull()).count()
print("DF_AQ_2 - Número de filas con time_format null:", null_count)

+----+-----+----+---+---+--------+-----------+---+-----------+
|time|pm2_5|pm10| co| o3|humidity|temperature|pm5|time_format|
+----+-----+----+---+---+--------+-----------+---+-----------+
+----+-----+----+---+---+--------+-----------+---+-----------+



DF_AQ_2 - Número de filas con time_format null: 0


In [148]:
#despues de verificar que la columna time_format no tiene valores nulos se procede a eliminar la columna time
DF_AQ_SEP = DF_AQ_SEP.drop("time")
DF_AQ_SEP = DF_AQ_SEP.drop("pm5")
DF_AQ_1 = DF_AQ_1.drop("time")
DF_AQ_2 = DF_AQ_2.drop("time")

In [149]:
DF_AQ_SEP = DF_AQ_SEP.withColumn("pm2_5",
    when(col("pm2_5") <= -990, None).otherwise(col("pm2_5"))
)
DF_AQ_SEP = DF_AQ_SEP.withColumn("pm10",
    when(col("pm10") <= -990, None).otherwise(col("pm10"))
)
DF_AQ_SEP = DF_AQ_SEP.withColumn("co",
    when(col("co") <= -990, None).otherwise(col("co"))
)
DF_AQ_SEP = DF_AQ_SEP.withColumn("o3",
    when(col("o3") <= -990, None).otherwise(col("o3"))
)
DF_AQ_SEP = DF_AQ_SEP.withColumn("humidity",
    when(col("humidity") <= -990, None).otherwise(col("humidity"))
)
DF_AQ_SEP = DF_AQ_SEP.withColumn("temperature",
    when(col("temperature") <= -990, None).otherwise(col("temperature"))
)

In [150]:
DF_AQ_1 = DF_AQ_1.withColumn("pm2_5",
    when(col("pm2_5") <= -990, None).otherwise(col("pm2_5"))
)
DF_AQ_1 = DF_AQ_1.withColumn("pm10",
    when(col("pm10") <= -990, None).otherwise(col("pm10"))
)
DF_AQ_1 = DF_AQ_1.withColumn("co",
    when(col("co") <= -990, None).otherwise(col("co"))
)
DF_AQ_1 = DF_AQ_1.withColumn("o3",
    when(col("o3") <= -990, None).otherwise(col("o3"))
)
DF_AQ_1 = DF_AQ_1.withColumn("humidity",
    when(col("humidity") <= -990, None).otherwise(col("humidity"))
)
DF_AQ_1 = DF_AQ_1.withColumn("temperature",
    when(col("temperature") <= -990, None).otherwise(col("temperature"))
)
DF_AQ_1 = DF_AQ_1.withColumn("pm5",
    when(col("pm5") <= -990, None).otherwise(col("pm5"))
)

In [151]:
DF_AQ_2 = DF_AQ_2.withColumn("pm2_5",
    when(col("pm2_5") <= -990, None).otherwise(col("pm2_5"))
)
DF_AQ_2 = DF_AQ_2.withColumn("pm10",
    when(col("pm10") <= -990, None).otherwise(col("pm10"))
)
DF_AQ_2 = DF_AQ_2.withColumn("co",
    when(col("co") <= -990, None).otherwise(col("co"))
)
DF_AQ_2 = DF_AQ_2.withColumn("o3",
    when(col("o3") <= -990, None).otherwise(col("o3"))
)
DF_AQ_2 = DF_AQ_2.withColumn("humidity",
    when(col("humidity") <= -990, None).otherwise(col("humidity"))
)
DF_AQ_2 = DF_AQ_2.withColumn("temperature",
    when(col("temperature") <= -990, None).otherwise(col("temperature"))
)
DF_AQ_2 = DF_AQ_2.withColumn("pm5",
    when(col("pm5") <= -990, None).otherwise(col("pm5"))
)

In [152]:
# FILTRAR FILAS CON VALORES NULOS EN COLUMNAS DIFERENTES AL TIME
cols = ["pm2_5", "pm10", "co", "o3", "humidity", "temperature","pm5"]
# Construir la condición: cada columna debe ser null
condition = reduce(lambda acc, c: acc & col(c).isNull(), cols, lit(True))
# Filtrar el DataFrame
DF_AQ_SEP_null_rows = DF_AQ_SEP.filter(condition)
DF_AQ_2_null_rows = DF_AQ_2.filter(condition)
DF_AQ_1_null_rows = DF_AQ_1.filter(condition)
# Mostrar el resultado
print(DF_AQ_SEP_null_rows.count())
print(DF_AQ_1_null_rows.count())
print(DF_AQ_2_null_rows.count())

53


51
50


In [153]:
#AHORA SE ELIMINAN LAS FILAS CON VALORES NULOS
# Lista de columnas a verificar
cols = ["pm2_5", "pm10", "co", "o3", "humidity", "temperature","pm5"]

# Construir la condición que identifica filas donde todas las columnas son null
condition_all_null = reduce(lambda acc, c: acc & col(c).isNull(), cols, lit(True))

# Filtrar el DataFrame eliminando aquellas filas que cumplan la condición
DF_AQ_SEP = DF_AQ_SEP.filter(~condition_all_null)
DF_AQ_1 = DF_AQ_1.filter(~condition_all_null)
DF_AQ_2 = DF_AQ_2.filter(~condition_all_null)

In [169]:
# Variables globales
engine = None

def connect_to_db():
    global engine
    user = 'root'
    password = 'rootpassword'
    host = '127.0.0.1'
    port = 3309
    db = 'AIRQUALITY'

    connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}"
    engine = create_engine(connection_string)
    print("✅ Conexión a MySQL creada.")


In [170]:
def create_tables_if_not_exist():
    with engine.connect() as conn:
        # Tabla DF_AQ_SEP
        conn.execute(text("""
            CREATE TABLE IF NOT EXISTS DF_AQ_SEP (
                pm2_5 DOUBLE,
                pm10 DOUBLE,
                co DOUBLE,
                o3 DOUBLE,
                humidity DOUBLE,
                temperature DOUBLE,
                time_format VARCHAR(50)
            )
        """))

        # Tabla DF_AQ_1
        conn.execute(text("""
            CREATE TABLE IF NOT EXISTS DF_AQ_1 (
                pm2_5 INT,
                pm10 INT,
                co INT,
                o3 INT,
                humidity DOUBLE,
                temperature DOUBLE,
                pm5 INT,
                time_format VARCHAR(50)
            )
        """))

        # Tabla DF_AQ_2
        conn.execute(text("""
            CREATE TABLE IF NOT EXISTS DF_AQ_2 (
                pm2_5 INT,
                pm10 INT,
                co INT,
                o3 INT,
                humidity DOUBLE,
                temperature DOUBLE,
                pm5 INT,
                time_format VARCHAR(50)
            )
        """))

        print("✅ Tablas verificadas y creadas si era necesario.")


In [176]:
def insert_dataframe(df, table_name):
    if engine is None:
        raise Exception("Primero debes establecer la conexión con `connect_to_db()`.")

    try:
        df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        print(f"✅ Datos insertados en la tabla {table_name}.")
    except Exception as e:
        print(f"❌ Error al insertar datos en la tabla {table_name}: {e}")


In [177]:
connect_to_db()
create_tables_if_not_exist()

# Insertar los dataframes limpiando primero cada tabla
insert_dataframe(DF_AQ_SEP.toPandas(), "DF_AQ_SEP")
insert_dataframe(DF_AQ_1.toPandas(), "DF_AQ_1")
insert_dataframe(DF_AQ_2.toPandas(), "DF_AQ_2")


✅ Conexión a MySQL creada.
✅ Tablas verificadas y creadas si era necesario.
✅ Datos insertados en la tabla DF_AQ_SEP.


✅ Datos insertados en la tabla DF_AQ_1.


✅ Datos insertados en la tabla DF_AQ_2.


In [178]:
DF_AQ_SEP.show()

+-----+----+----+---+--------+-----------+-------------------+
|pm2_5|pm10|  co| o3|humidity|temperature|        time_format|
+-----+----+----+---+--------+-----------+-------------------+
|  0.5| 0.5|25.3|0.1|    NULL|       11.5|2024-03-15 00:11:34|
|  0.7| 0.7|23.0|0.1|    NULL|       11.6|2024-03-15 00:26:34|
|  0.7| 0.7|22.2|0.1|    NULL|       11.2|2024-03-15 00:41:34|
|  0.6| 0.6|20.4|0.1|    NULL|       11.1|2024-03-15 00:56:34|
|  0.6| 0.6|19.6|0.1|    NULL|       10.6|2024-03-15 01:11:34|
|  0.5| 0.5|19.2|0.1|    NULL|       10.6|2024-03-15 01:26:34|
|  0.7| 0.7|19.5|0.1|    NULL|       10.4|2024-03-15 01:41:34|
|  0.6| 0.6|21.2|0.1|    NULL|       10.5|2024-03-15 01:56:34|
|  0.5| 0.5|20.3|0.1|    NULL|       10.1|2024-03-15 02:11:34|
|  0.6| 0.6|19.2|0.1|    NULL|        9.9|2024-03-15 02:26:34|
|  0.5| 0.5|19.7|0.1|    NULL|        9.7|2024-03-15 02:41:34|
|  0.4| 0.4|17.7|0.1|    NULL|        9.1|2024-03-15 02:56:34|
|  0.6| 0.6|17.0|0.1|    NULL|        9.1|2024-03-15 03